In [1]:
%%javascript
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false; // disable scrolling
}

<IPython.core.display.Javascript object>

In [2]:
# Google Forms / Sheets Code
import urllib.request
from bs4 import BeautifulSoup
import requests, warnings
def get_questions(in_url):
    res = urllib.request.urlopen(in_url)
    soup = BeautifulSoup(res.read(), 'html.parser')
    get_names = lambda f: [v for k,v in f.attrs.items() if 'label' in k]
    get_name = lambda f: get_names(f)[0] if len(get_names(f))>0 else 'unknown'
    all_questions = soup.form.findChildren(attrs={'name': lambda x: x and x.startswith('entry.')})
    return {get_name(q): q['name'] for q in all_questions}
def submit_response(form_url, cur_questions, verbose=False, **answers):
    submit_url = form_url.replace('/viewform', '/formResponse')
    form_data = {'draftResponse':[],
                'pageHistory':0}
    for v in cur_questions.values():
        form_data[v] = ''
    for k, v in answers.items():
        if k in cur_questions:
            form_data[cur_questions[k]] = v
        else:
            warnings.warn('Unknown Question: {}'.format(k), RuntimeWarning)
    if verbose:
        print(form_data)
    user_agent = {'Referer':form_url,
                  'User-Agent': "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.52 Safari/537.36"}
    return requests.post(submit_url, data=form_data, headers=user_agent)

In [3]:
import os
import ipywidgets as ipw
from glob import glob
import json
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle
from io import BytesIO
from time import time
USERNAME=os.environ.get('APPMODE_USER', 'Not logged in')
HOSTNAME=os.environ.get('HOSTNAME', 'anon')
BUTTONS_PER_ROW = 3
BUTTON_WIDTH = "150px"
UNKNOWN_OPTION = 'Unknown'
SINGLE_CATEGORY_MODE = True
if SINGLE_CATEGORY_MODE:
    BUTTONS_PER_ROW = 1

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
with open('task.json', 'r') as f:
    annotation_task = json.load(f)
    annot_df = pd.DataFrame(annotation_task['dataset']['dataframe'])
print('Loaded annotation task', annot_df.shape[0], 'images')

Loaded annotation task 110 images


In [5]:
image_path_dict = {c_path: os.path.join(annotation_task['dataset']['base_image_directory'], 
                                    c_path)
                                    for c_path in annot_df[annotation_task['dataset']['image_path']]}
category_ids = annot_df[annotation_task['dataset']['output_labels']].unique().tolist()
image_keys = list(image_path_dict.keys())
if SINGLE_CATEGORY_MODE:
    button_ids = ['Yes', 'No']
    def get_next_image_question_pair(seed=None):
        if seed is not None:
            np.random.seed(seed)
        question = 'Does the image show signs of {}?'.format(np.random.choice(category_ids))
        return question, np.random.choice(image_keys)
else:    
    button_ids = category_ids
    def get_next_image_question_pair(seed=None):
        if seed is not None:
            np.random.seed(seed)
        return 'Select the most appropriate label for the given image', np.random.choice(image_keys)

if UNKNOWN_OPTION is not None:
    button_ids += [UNKNOWN_OPTION]    

In [6]:
FORM_URL = annotation_task['google_forms']['form_url']
anno_questions = get_questions(FORM_URL)

In [7]:
global out_rows
out_rows = []
def submit_label(label_id, label_name):
    """
    should be linked to some kind of database
    """
    global out_rows
    print('Submitting', label_name, 'for', label_id, 'from', USERNAME, 'part of', HOSTNAME, '@', time())
    submit_response(FORM_URL, anno_questions, 
                    annotator=USERNAME, 
                    session=HOSTNAME, 
                    time = time(),
                    item_id = label_id,
                    label = label_name,
                    task = 'pneumonia_annotation'
                   )
    out_rows += [dict(label_name=label_name, label_id=label_id, user=USERNAME, session=HOSTNAME, time=time())]
    return pd.DataFrame(out_rows).to_html()

In [12]:
title_box = ipw.HTML(value=f'<h1> Welcome {USERNAME}</h1>')
question_box = ipw.HTML(value='')
cur_image_view = ipw.Image(layout=ipw.Layout(width="512px"),  
                  disabled=True)
cur_img_id = ipw.Text(layout=ipw.Layout(width="64px"),  
                  disabled=True)
annot_results = ipw.HTML(value='')

SESSION_ID = None

def get_next_image():
    c_label, c_key = get_next_image_question_pair()
    
    question_box.value='<h2>{}</h2>'.format(c_label)
    cur_img_id.value = c_key
    c_img = Image.open(image_path_dict[c_key])
    bio_obj = BytesIO()
    c_img.save(bio_obj, format='png')
    bio_obj.seek(0)
    cur_image_view.value=bio_obj.read()

def on_click(btn):
    if btn is not None:
        annot_results.value = submit_label(cur_img_id.value, btn.description)
    get_next_image()
        

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width=BUTTON_WIDTH))
    if description=='Yes':
        btn.style.button_color = 'lightgreen'
    elif description=='No':
        btn.style.button_color = 'pink'
    btn.on_click(on_click)
    return btn

button_rows = []
c_row = []
for i, but_name in enumerate(button_ids, 1):
    c_row+=[mk_btn(but_name)]
    if (i % BUTTONS_PER_ROW)==0:
        button_rows+=[ipw.HBox(c_row)]
        c_row=[]
button_rows+=[ipw.HBox(c_row)]

on_click(None) # initialize

annot_view = [title_box, 
          ipw.HBox([cur_image_view, ipw.VBox([question_box]+button_rows)])]

ipw.VBox(annot_view)

Submitting No for 00017199_006.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150433.7154396
Submitting Unknown for 00030323_030.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150441.95584
Submitting Yes for 00020259_001.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150446.9748096
Submitting Yes for 00020259_001.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150448.8481047
Submitting Yes for 00013128_010.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150450.2742739
Submitting Yes for 00000032_016.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150452.090327
Submitting Yes for 00020791_002.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150453.74455
Submitting Yes for 00015072_003.png from dr_head part of jupyter-chestrays-2djupyanno-2d7x7uuhxf @ 1534150456.5051274
Submitting No for 00002224_007.png from dr_head part of ju